In [ ]:
!python -m pip install azure-ai-vision
!python -m pip install --upgrade azure-ai-vision

In [ ]:
!pip install azure-ai-vision
!pip install azure-identity

In [ ]:
!pip install python-dotenv
!pip install openai

In [2]:
import os
import azure.ai.vision as sdk
import json

service_options = sdk.VisionServiceOptions(os.environ["VISION_ENDPOINT"],
                                           os.environ["VISION_KEY"])

In [3]:
print(service_options)

# 이미지 파일 입력 시, gpt3.5 사용한 최종 대체텍스트 결과 출력하는 코드


In [7]:
import os, json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

def print_alt_text(subject) :

    # 1. 이미지 불러오기 
    image_path = 'image/{0}.jpg'.format(subject)
    with open(image_path, 'rb') as f:
        image_buffer = f.read()

    # 2. Azure Image analysis(captioning) API 사용하기
    image_source_buffer = sdk.ImageSourceBuffer()
    image_source_buffer.image_writer.write(image_buffer)
    vision_source = sdk.VisionSource(image_source_buffer=image_source_buffer)

    analysis_options = sdk.ImageAnalysisOptions()
    analysis_options.features = (
        sdk.ImageAnalysisFeature.DENSE_CAPTIONS |
        sdk.ImageAnalysisFeature.TEXT
    )

    analysis_options.language = "en"
    analysis_options.gender_neutral_caption = True

    # API 호출
    image_analyzer = sdk.ImageAnalyzer(service_options, vision_source, analysis_options) 
    result = image_analyzer.analyze()
    if result.reason == sdk.ImageAnalysisResultReason.ANALYZED:
        if result.dense_captions is not None: # Image Caption 결과
            print(" Dense Captions:")
            caption_li = []
            for caption in result.dense_captions:
                print("   '{}', {}, Confidence: {:.4f}".format(caption.content, caption.bounding_box, caption.confidence))
                caption_li.append(caption.content)

        if result.text is not None: # Image OCR 결과
            print(" Text:")
            text_li = []
            for line in result.text.lines:
                points_string = "{" + ", ".join([str(int(point)) for point in line.bounding_polygon]) + "}"
                print("   Line: '{}', Bounding polygon {}".format(line.content, points_string))
                text_li.append(line.content)
                # for word in line.words:
                #     points_string = "{" + ", ".join([str(int(point)) for point in word.bounding_polygon]) + "}"
                #     # print("     Word: '{}', Bounding polygon {}, Confidence {:.4f}"
                #     #       .format(word.content, points_string, word.confidence))

    else:
        error_details = sdk.ImageAnalysisErrorDetails.from_result(result)
        print(" Analysis failed.")
        print("   Error reason: {}".format(error_details.reason))
        print("   Error code: {}".format(error_details.error_code))
        print("   Error message: {}".format(error_details.message))

    text_ext =  ' '.join(text_li)
    dic = {'caption' : caption_li, 'ocr' : text_ext }

    # 3. Image 캡션과 OCR결과 json파일로 저장하기
    with open('fromocr/{0}.json'.format(subject), 'w', encoding='utf-8') as f:
        f.write(json.dumps(dic, ensure_ascii=False, indent=4))

    # 4. OCR 결과 불러오기
    json_file_path = r"fromocr\{0}.json".format(subject)
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # 5. AzureOpenAI API 호출하기
    client = AzureOpenAI(
        # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
        api_version="2023-07-01-preview",
        # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
        azure_endpoint="",
        api_key = os.environ.get("AZURE_OPENAI_API_KEY"),
    )

    completion = client.chat.completions.create(
        model="gpt-35-instant",  # e.g. gpt-35-instant 
        messages=[
            {
                "role": "user",
                "content": str(data) + 
                """ 위 딕셔너리의 caption의 value값은 이미지의 그림을 설명하는 것이고, ocr의 value값은 OCR된 이미지에 나와있는 모든 텍스트야.
                너는 이 둘을 전체적으로 종합해서 존댓말로 시각장애인한테 설명해야하는 상황이야. 
                단,OCR의 VALUE값의 길이가 200자 이상일 경우이거나 이미지의 주요 물체가 식품일 경우, ocr value값이 caption value값보다 더 중요해. 
                그럴 경우에는, ocr value값에서 사람이 반드시 알아야하는 정보 최소 3개~5개를 말해줘. 식품의 영양정보는 반드시 정보 그대로 말해줘.
                또는, OCR의 VALUE값의 길이가 200자 미만일 경우이거나  이미지의 주요 물체가 의류일 경우 또는 다양한 색상이 있는 경우, caption value값에 초점을 맞춰서 전체적인 모습을 색상, 질감, 모양, 형태에 대해  전체 답변을 2~3문장내로 풍부하게 묘사해줘. 또 조건이 있어. 단, 너의 말에 OCR이라는 단어는 절대 넣지말고, 네, 알겠습니다. 이런 말은 하지말아줘.
    """,
            },
        ],
    )
    print(completion.model_dump_json(indent=2))
    print(completion.choices[0].message.content)

    txt_file_path = r"result\{0}3.5.txt".format(subject)
    with open(txt_file_path, 'w') as file:
        file.write(completion.choices[0].message.content)


print_alt_text("snow") 

 Dense Captions:
   'a snow covered tree and cars', Rectangle(x=0, y=0, w=1054, h=535), Confidence: 0.7152
   'a white car parked in a parking lot', Rectangle(x=0, y=267, w=350, h=166), Confidence: 0.8197
   'a white car parked in a parking lot', Rectangle(x=176, y=249, w=260, h=130), Confidence: 0.7094
   'a snow covered tree branch', Rectangle(x=409, y=285, w=424, h=239), Confidence: 0.6705
   'a snow on a tree', Rectangle(x=710, y=244, w=253, h=98), Confidence: 0.6877
 Text:
   Line: 'YONHAP NEWS', Bounding polygon {786, 464, 1045, 463, 1045, 495, 786, 496}
{
  "id": "chatcmpl-8UmNpULosh3ywbYn85XznG9GuN9rQ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "이 이미지들은 겨울 풍경을 포착한 것으로 보입니다. 눈으로 덮인 나무와 나뭇가지들이 얼음의 무게로 굽어 있는 모습이 인상적이고, 고요하고 차가운 분위기를 자아내고 있습니다. 또한, 흰색 차량이 주차장에 덮힌 새하얀 눈과 대조를 이뤄 서 있습니다. 사진 속에는 'YONHAP NEWS'라는 텍스트도 확인되는데, 이는 이미지에 포함된 텍스트 정보입니다.",
        "role": "assistant",
        "function_call": null,
        "tool_